In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [2]:
train = pd.read_csv('D:/ML/Kaggle/playground-series-s5e7/train.csv')
test = pd.read_csv('D:/ML/Kaggle/playground-series-s5e7/test.csv')

In [4]:
report = ProfileReport(train)

In [5]:
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 383.75it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
test['Time_spent_Alone']= test['Time_spent_Alone'].fillna(test['Time_spent_Alone'].median())
test['Stage_fear']= test['Stage_fear'].fillna(test['Stage_fear'].mode())
test['Social_event_attendance'] = test['Social_event_attendance'].fillna(test['Social_event_attendance'].median())
test['Going_outside'] = test['Going_outside'].fillna(test['Going_outside'].median())
test['Drained_after_socializing'] = test['Drained_after_socializing'].fillna(test['Drained_after_socializing'].mode())
test['Friends_circle_size'] = test['Friends_circle_size'].fillna(test['Friends_circle_size'].median())
test['Post_frequency'] = test['Post_frequency'].fillna(test['Post_frequency'].median())

In [20]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGeneralizedLinearEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,23 secs
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,8 months and 24 days
H2O_cluster_name:,H2O_from_python_mhkir_rmvyha
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.726 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
df = h2o.H2OFrame(train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
df_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
df_train, df_valid  = df.split_frame(ratios=[.7], seed=123)

In [10]:
model_gbm = H2OGradientBoostingEstimator(nfolds=5, 
                                         seed=1111, 
                                         ntrees=50,
                                         keep_cross_validation_predictions=True)

In [11]:
model_rf = H2ORandomForestEstimator(nfolds=5, 
                                    ntrees=50,
                                    seed=1111, 
                                    keep_cross_validation_predictions=True)

In [18]:
model_glm = H2OGeneralizedLinearEstimator(nfolds=5, 
                                          family='binomial',
                                          compute_p_values=True, 
                                          remove_collinear_columns=True,
                                          keep_cross_validation_predictions= True)

In [19]:
df_train.types


{'id': 'int',
 'Time_spent_Alone': 'int',
 'Stage_fear': 'enum',
 'Social_event_attendance': 'int',
 'Going_outside': 'int',
 'Drained_after_socializing': 'enum',
 'Friends_circle_size': 'int',
 'Post_frequency': 'int',
 'Personality': 'enum'}

In [13]:
response = 'Personality'
predictors = ['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size', 'Post_frequency' ]

In [16]:
## Variables 
train.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,5.0,Extrovert


In [14]:
model_gbm.train(x = predictors, y = response, training_frame=df_train, validation_frame=df_valid)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1753501454093_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          16477                  5            5            5             16            29            21.64

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.027671761445636097
RMSE: 0.1663483136242628
LogLoss: 0.12038684046459096
Mean Per-Class Error: 0.0401806379972751
AUC: 0.9797324429893733
AUCPR: 0.9601165002873598
Gini: 0.9594648859787467

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.45766709030300723
           Extrovert    Introvert    Error    Rate
---------  -----------  -----------  -------  ---------------
Extrovert  9463         181          0.0188   (181.0/9644.0)
Introvert  208          3169         0.0616   (208.0/3377.0)
Total      9671         3350         0.0299   (389.0/13021.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.457667     0.942173  202
max f2                       0.226111     0.941845  237
max f0point5                 0.735938     0.94571   170
max accuracy                 0.572731     0.970125  193
max precision                0.989108     1         0
max recall                   0.0113265    1         395
max specificity              0.989108     1         0
max absolute_mcc             0.457667     0.922047  202
max min_per_class_accuracy   0.0528395    0.95114   294
max mean_per_class_accuracy  0.295856     0.960515  223
max tns                      0.989108     9644      0
max fns                      0.989108     3375      0
max fps                      0.0098624    9644      399
max tps                      0.0113265    3377      395
max tnr                      0.989108     1         0
max fnr                      0.989108     0.999408  0
max fpr                      0.0098624    1         399
max tpr                      0.0113265    1         395

Gains/Lift Table: Avg response rate: 25.94 %, avg score: 25.94 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100607                   0.972015           3.82636     3.82636            0.992366         0.975492   0.992366                    0.975492            0.0384957       0.0384957                  282.636   282.636            0.038392
2        0.0202749                   0.968196           3.85579     3.84118            1                0.969742   0.996212                    0.972595            0.0393841       0.0778798                  285.579   284.118            0.0777761
3        0.0301052                   0.96628            3.85579     3.84595            1                0.967251   0.997449                    0.97085             0.0379035       0.115783                   285.579   284.595            0.11568
4        0.0400123                   0.963663           3.8259      3.84099            0.992248         0.964801   0.996161                    0.969352            0.0379035       0.153687                   282.59    284.099            0.153479
5        0.050073                    0.9608

In [15]:
model_rf.train(x = predictors, y = response, training_frame=df_train, validation_frame=df_valid)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1753501454093_620


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          437813                 20           20           20            511           781           698.58

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.03386038059798578
RMSE: 0.1840119034138438
LogLoss: 0.3391780572885399
Mean Per-Class Error: 0.044504787982530464
AUC: 0.9621770136798975
AUCPR: 0.9219233765355624
Gini: 0.924354027359795

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.44689847921070297
           Extrovert    Introvert    Error    Rate
---------  -----------  -----------  -------  ---------------
Extrovert  9431         213          0.0221   (213.0/9644.0)
Introvert  226          3151         0.0669   (226.0/3377.0)
Total      9657         3364         0.0337   (439.0/13021.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.446898     0.934876  197
max f2                       0.305051     0.935343  218
max f0point5                 0.604107     0.937444  172
max accuracy                 0.463184     0.966285  196
max precision                0.999999     0.95537   0
max recall                   1.07781e-06  1         399
max specificity              0.999999     0.996682  0
max absolute_mcc             0.446898     0.912134  197
max min_per_class_accuracy   0.162184     0.942553  264
max mean_per_class_accuracy  0.389265     0.95585   204
max tns                      0.999999     9612      0
max fns                      0.999999     2692      0
max fps                      1.07781e-06  9644      399
max tps                      1.07781e-06  3377      399
max tnr                      0.999999     0.996682  0
max fnr                      0.999999     0.797157  0
max fpr                      1.07781e-06  1         399
max tpr                      1.07781e-06  1         399

Gains/Lift Table: Avg response rate: 25.94 %, avg score: 25.98 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0549881                   1                  3.68346    3.68346            0.955307         1            0.955307                    1                   0.202547        0.202547                   268.346   268.346            0.199229
2        0.100069                    0.99               3.65873    3.67232            0.948893         0.995306     0.952417                    0.997885            0.164939        0.367486                   265.873   267.232            0.361057
3        0.150065                    0.950353           3.66033    3.66833            0.949309         0.973808     0.951382                    0.989863            0.183003        0.550489                   266.033   266.833            0.540638
4        0.200061                    0.9                3.63664    3.66041            0.943164         0.92944      0.949328                    0.974763            0.181818        0.732307                   263.664   266.041            0.71862
5        0.300054                    0.1

In [19]:
model_glm.train(x = predictors, y = response, training_frame=df_train, validation_frame=df_valid)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_model_python_1753501454093_1235


GLM Model: summary
    family    link    regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 6.329E-4 )  11                            9                              5                       py_2_sid_93d6

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.03125729287497618
RMSE: 0.17679732145871493
LogLoss: 0.14348930380180103
AUC: 0.9584399929156994
AUCPR: 0.9061575240368361
Gini: 0.9168799858313987
Null degrees of freedom: 13020
Residual degrees of freedom: 13011
Null deviance: 14905.821863767744
Residual deviance: 3736.7484496065026
AIC: 3756.7484496065026

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.40600183229980397
           Extrovert    Introvert    Error    Rate
---------  -----------  -----------  -------  ---------------
Extrovert  9448         196          0.0203   (196.0/9644.0)
Introvert  219          3158         0.0649   (219.0/3377.0)
Total      9667         3354         0.0319   (415.0/13021.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.406002     0.938345  193
max f2                       0.267432     0.936537  203
max f0point5                 0.669341     0.940653  183
max accuracy                 0.406002     0.968128  193
max precision                0.844108     0.943831  151
max recall                   0.00571479   1         398
max specificity              0.987224     0.999896  0
max absolute_mcc             0.406002     0.916865  193
max min_per_class_accuracy   0.0782551    0.939887  259
max mean_per_class_accuracy  0.406002     0.957413  193
max tns                      0.987224     9643      0
max fns                      0.987224     3373      0
max fps                      0.0049188    9644      399
max tps                      0.00571479   3377      398
max tnr                      0.987224     0.999896  0
max fnr                      0.987224     0.998816  0
max fpr                      0.0049188    1         399
max tpr                      0.00571479   1         398

Gains/Lift Table: Avg response rate: 25.94 %, avg score: 13.80 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100607                   0.213338           1.08904   1.08904            0.282443         0.227402   0.282443                    0.227402            0.0109565       0.0109565                  8.90397   8.90397            0.00120948
2        0.0200445                   0.203603           0.800818  0.945481           0.207692         0.207841   0.245211                    0.217659            0.00799526      0.0189517                  -19.9182  -5.45191           -0.00147548
3        0.0300284                   0.197152           1.483     1.12419            0.384615         0.200351   0.29156                     0.211905            0.014806        0.0337578                  48.2996   12.4194            0.00503525
4        0.0400123                   0.19198            1.60164   1.24333            0.415385         0.1

In [21]:
perf= model.model_performance()
perf

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.027871182575591617
RMSE: 0.1669466458950033
LogLoss: 0.12126223393597033
Mean Per-Class Error: 0.0398182093300288
AUC: 0.9784037374598483
AUCPR: 0.963066528584311
Gini: 0.9568074749196966

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.41473082148993695
           Extrovert    Introvert    Error    Rate
---------  -----------  -----------  -------  ---------------
Extrovert  9450         194          0.0201   (194.0/9644.0)
Introvert  201          3176         0.0595   (201.0/3377.0)
Total      9651         3370         0.0303   (395.0/13021.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.414731     0.941455  242
max f2                       0.180847     0.942063  267
max f0point5                 0.837352     0.945094  180
max accuracy                 0.414731     0.969664  242
max precision                0.984985     1         0
max recall                   0.0133552    1         391
max specificity              0.984985     1         0
max absolute_mcc             0.414731     0.920985  242
max min_per_class_accuracy   0.0449335    0.949956  324
max mean_per_class_accuracy  0.235476     0.960581  260
max tns                      0.984985     9644      0
max fns                      0.984985     3376      0
max fps                      0.00750905   9644      399
max tps                      0.0133552    3377      391
max tnr                      0.984985     1         0
max fnr                      0.984985     0.999704  0
max fpr                      0.00750905   1         399
max tpr                      0.0133552    1         391

Gains/Lift Table: Avg response rate: 25.94 %, avg score: 25.93 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100607                   0.974899           3.85579    3.85579            1                0.977808   1                           0.977808            0.0387918       0.0387918                  285.579   285.579            0.0387918
2        0.0201213                   0.969515           3.82636    3.84107            0.992366         0.972109   0.996183                    0.974958            0.0384957       0.0772875                  282.636   284.107            0.0771838
3        0.0301052                   0.966076           3.85579    3.84595            1                0.967754   0.997449                    0.972569            0.0384957       0.115783                   285.579   284.595            0.11568
4        0.0400123                   0.963564           3.8259     3.84099            0.992248         0.964708   0.996161                    0.970623            0.0379035       0.153687                   282.59    284.099            0.153479
5        0.050073                    0.961422           3.85579    3.84396            1                0.962375   0.996933                    0.968966            0.0387918       0.192479                   285.579   284.396            0.192271
6        0.100069                    0.950472           3.79656    3.82028            0.984639         0.955827   0.99079                     0.962401            0.189813        0.382292                   279.656   282.028            0.381048
7        0.150219                    0.939369           3.77903    3.80651            0.980092         0.944829   0.987219                    0.956535            0.18951

In [21]:
ensemble = H2OStackedEnsembleEstimator(model_id='ensemble_model', 
                                       base_models=[model_gbm, model_rf, model_glm])

In [22]:
ensemble.train(x = predictors, y = response, training_frame=df_train, validation_frame=df_valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/3
# GBM base models (used / total),1/1
# DRF base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [ ]:
test

In [25]:
pred = ensemble.predict(df_test)
pred

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Extrovert,Introvert
Extrovert,0.979354,0.020646
Introvert,0.113301,0.886699
Extrovert,0.973302,0.0266984
Extrovert,0.97895,0.0210501
Introvert,0.0448387,0.955161
Extrovert,0.979365,0.0206345
Extrovert,0.977134,0.0228664
Introvert,0.0471083,0.952892
Extrovert,0.979685,0.0203148
Introvert,0.0456917,0.954308


In [26]:
pred_df = pred.as_data_frame()

d:\anaconda\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [27]:
pred_df['id'] = test['id']

In [28]:
pred_df[['id', 'predict']].to_csv('submission2.csv', index = False)

In [29]:
import os
print(os.getcwd())

d:\ML\Kaggle\playground-series-s5e7
